In [ ]:
# Google Colab에서 Kaggle API를 사용하기 위한 라이브러리 설치
!pip install kaggle

# Kaggle API 토큰 업로드
from google.colab import files
files.upload()

In [ ]:
# Kaggle API 토큰을 사용자 디렉토리로 복사하고 권한 설정
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Kaggle에서 데이터셋 다운로드
!kaggle datasets download -d hetulmehta/website-classification

In [ ]:
# 다운로드한 데이터셋 압축 해제
!unzip website-classification.zip

In [ ]:
# 필요한 라이브러리 및 모듈 로드
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [ ]:
# 데이터 로드 및 전처리
wc_data = pd.read_csv('website_classification.csv')
# 'Streaming Services' 카테고리를 1로, 다른 카테고리를 0으로 변환
wc_data['Outlook'] = (wc_data['Category'] == 'Streaming Services').astype(int)
# 훈련 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(wc_data['cleaned_website_text'], wc_data['Outlook'], test_size=0.2, random_state=42)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip glove*.zip

In [ ]:
!ls
!pwd

In [ ]:
import os

file_path = 'glove.6B.50d.txt'

if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
else:
    print(f"The file '{file_path}' does not exist. Please check the path.")

In [ ]:
import os

print("Current Working Directory:", os.getcwd())

In [ ]:
from gensim.models import KeyedVectors

# GloVe 워드 임베딩 모델 로드
glove_model = KeyedVectors.load_word2vec_format('glove.6B.50d.txt', binary=False, no_header=True)

In [ ]:
# 텍스트 데이터 토큰화
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(wc_data['cleaned_website_text'])
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

In [ ]:
# 시퀀스를 고정 길이로 패딩
max_len = 100
X_train_padded = pad_sequences(X_train_tokens, maxlen=max_len)
X_test_padded = pad_sequences(X_test_tokens, maxlen=max_len)

In [ ]:
# 토큰을 워드 임베딩 벡터로 변환
X_train_vectors = np.zeros((len(X_train_padded), max_len, 50))
X_test_vectors = np.zeros((len(X_test_padded), max_len, 50))

# 정수 인덱스를 단어로 변환하기 위한 역 매핑
reverse_word_index = {index: word for word, index in tokenizer.word_index.items()}

# 훈련 데이터에 대한 워드 임베딩 벡터 생성
for i, seq in enumerate(X_train_padded):
    for j, index in enumerate(seq):
        word = reverse_word_index.get(index, '')  # 인덱스에 해당하는 단어 가져오기
        if word in glove_model:
            X_train_vectors[i][j] = glove_model[word]

# 테스트 데이터에 대한 워드 임베딩 벡터 생성
for i, seq in enumerate(X_test_padded):
    for j, index in enumerate(seq):
        word = reverse_word_index.get(index, '')  # 인덱스에 해당하는 단어 가져오기
        if word in glove_model:
            X_test_vectors[i][j] = glove_model[word]

In [ ]:
# 워드 임베딩 매트릭스 생성
embedding_dim = 50
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in glove_model:
        embedding_matrix[i] = glove_model[word]

In [ ]:
!pip install tensorflow-gpu

In [ ]:
import tensorflow as tf

# GPU 사용 가능 여부 확인
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print("TensorFlow-GPU is available and configured.")
else:
    print("No GPU detected. TensorFlow will use CPU.")

In [ ]:
# TensorFlow 업그레이드
!pip install --upgrade tensorflow

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Flatten

# LSTM 기반 신경망 모델 생성
model = Sequential()

# LSTM 레이어 추가
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, input_shape=(max_len, embedding_dim)))

# 이진 분류를 위한 Dense 레이어 추가
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 구조 출력
model.summary()

In [ ]:
# 모델 훈련
model.fit(X_train_vectors, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
# 테스트 및 평가
loss, accuracy = model.evaluate(X_test_vectors, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

In [ ]:
import matplotlib.pyplot as plt

# 훈련 기록 시각화
history = model.fit(X_train_vectors, y_train, epochs=10, batch_size=32, validation_split=0.2)

# 훈련 정확도 및 검증 정확도 그래프
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# 훈련 손실 및 검증 손실 그래프
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# 혼동 행렬 시각화
predictions = model.predict(X_test_vectors)
y_pred = (predictions > 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

# ROC 곡선 및 AUC 시각화
predictions = model.predict(X_test_vectors)
fpr, tpr, thresholds = roc_curve(y_test, predictions)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()